In [1]:
import pandas as pd
from datetime import datetime
import mysql.connector
import jdatetime
from sqlalchemy import create_engine
from urllib.parse import quote_plus as urlquote
from sqlalchemy.types import Integer, Text, String, DateTime, Numeric, Float

In [2]:

def what_time(reg):
    if (reg=="en"):
        return datetime.now()
    elif (reg=="fa"):
        return jdatetime.datetime.now()

    
def go_fetch_mysql(connection_engine, query_string=""):
    """
    Modern Way of Invoking Data From MySQL DB, Using sqlalchemy library
    
    Returns:
        Pandas DataFrame, including the result of entered query
    """
    if pd.io.sql._is_sqlalchemy_connectable(connection_engine)== True:
        df = pd.read_sql(query_string, con=connection_engine)
        connection_engine.dispose()
    else:
        return Exception 
    return df

def what_time_is_it(region="universal", granularity="date"):
    if region == "fa":
        fa_switch={
                    "date":jdatetime.datetime.now().strftime('%Y/%m/%d'),
                    "year_month":jdatetime.datetime.now().strftime('%Y_%m'),
                    "full":jdatetime.datetime.now()}
        return fa_switch.get(granularity,"___")
    
    elif region == "en":
        en_switch={
                    "date":datetime.now().strftime('%Y/%m/%d'),
                    "year_month":datetime.now().strftime('%Y_%m'),
                    "year_month_day":datetime.now().strftime('%Y%m%d'),
                    "full":datetime.now(),
                    "month_number":datetime.now().strftime('%m')}
        return en_switch.get(granularity,"___")
    elif region == "universal" and granularity == "date":
        return datetime.now().strftime('%H:%M')
    
    
def connection_builder_pgpool():
    host_address="10.0.0.102"
    port_number=5432
    username="postgres"
    connection_password="adminpassword"
    database_name="customdatabase"

    target_db_connection_str = ('postgresql+psycopg2://{0}:{1}@{2}:{3}/{4}'.format(username,connection_password,host_address,port_number,database_name))
    target_db_connection = create_engine(target_db_connection_str)
    return target_db_connection

def connection_builder_repmgr():
    host_address="10.0.0.102"
    port_number=49170
    username="repmgr"
    connection_password="repmgrpassword"
    database_name="customdatabase"

    target_db_connection_str = ('postgresql+psycopg2://{0}:{1}@{2}:{3}/{4}'.format(username,connection_password,host_address,port_number,database_name))
    target_db_connection = create_engine(target_db_connection_str)
    return target_db_connection


def write_this2mysql(connection_engine, dataframe_name, table_name, dup_startegy, target_table_structure):
    """
    
    Inserts Data from a Pandas DataFrame into Mysql DB
    you need specifying a dictionary including each of every table columns with their datatype in a dictionary,
    associated with target_table_structure argument
    
    for instance:
    
    sample={
    "radif": Integer,
    "ostan": Text,
    "markaz": Text,
    "tel":  Text}
    
    
    returns:
        nothing!
        
    Function Call Example:
    
    
    sample={
    "radif": Integer,
    "ostan": Text,
    "markaz": Text,
    "tel":  Text}
    
    Valid Data Types to consider: ["INTEGER","TEXT","VARCHAR(50)","DATETIME"]
    
    Strategy if the Table Name Exist:
        {‘fail’, ‘replace’, ‘replace’}, default ‘fail’
    
    write_this2mysql(dataframe_name=df,table_name='sample_table',dup_startegy='fail',target_table_structure=sample)
    
    """
    dataframe_name.to_sql(table_name, connection_engine,
        if_exists=dup_startegy, index=False,
        chunksize=500, dtype = target_table_structure)
    
    
def show_tables():
    r_set=db_connection.execute("SHOW TABLES");
    for row in r_set:
        print(row)

In [3]:
df = pd.read_excel("sample.xlsx")
df.head()

,RPT_DATE,VOL_UP,VOL_DOWN,CREATE_TIME
0,20200916,134300,262678,2020-09-17 06:39:32
1,20200916,144980896,152794608,2020-09-17 06:39:32
2,20200917,236948,1208641,2020-09-18 00:59:53
3,20200917,142089158,170982830,2020-09-18 00:59:53
4,20200918,283405,471215,2020-09-19 01:00:01


In [4]:
def sample_shit():
    res = pd.read_excel("sample.xlsx")
    dict_datatypes = {i:res.columns.get_loc(i) for i in res.columns}
    new_column_name = {'RPT_DATE':'date_of_being',
    'VOL_UP':'uploaded_data',
    'VOL_DOWN':'downloaded_data',
    'CREATE_TIME':'some_date'}
    
    res.rename(columns=new_column_name, inplace=True)

    dict_datatypes = {'date_of_being':String,
                        'uploaded_data(sec)':Float,
                        'downloaded_data(sec)':Float,
                      'some_date':DateTime}

    target_table_name = 'sample_table_{0}'.format(what_time_is_it(region="fa",granularity="year_month"))
    write_this2mysql(connection_engine=connection_builder_pgpool(),dataframe_name=res, table_name=target_table_name, dup_startegy='append', target_table_structure=dict_datatypes)
    
    target_query = "SELECT * FROM public.{0};".format(target_table_name)
    return go_fetch_mysql(connection_engine=connection_builder_pgpool(), query_string=target_query)

In [5]:
sample_shit()

,date_of_being,uploaded_data,downloaded_data,some_date
0,20200916,134300,262678,2020-09-17 06:39:32
1,20200916,144980896,152794608,2020-09-17 06:39:32
2,20200917,236948,1208641,2020-09-18 00:59:53
3,20200917,142089158,170982830,2020-09-18 00:59:53
4,20200918,283405,471215,2020-09-19 01:00:01
...,...,...,...,...
749,20210630,65984285,108621415,2021-06-30 12:30:01
750,20210630,5410279,5292425,2021-06-30 12:30:01
751,20210630,29300536,28373063,2021-06-30 12:30:01
752,20210630,2594045,2509931,2021-06-30 12:30:01


In [58]:
target_table_name = 'sample_table_{0}'.format(what_time_is_it(region="fa",granularity="year_month"))
target_query = "SELECT * FROM public.{0};".format(target_table_name)
go_fetch_mysql(connection_engine=connection_builder_repmgr(), query_string=target_query)

,date_of_being,uploaded_data,downloaded_data,some_date
0,20200916,134300,262678,2020-09-17 06:39:32
1,20200916,144980896,152794608,2020-09-17 06:39:32
2,20200917,236948,1208641,2020-09-18 00:59:53
3,20200917,142089158,170982830,2020-09-18 00:59:53
4,20200918,283405,471215,2020-09-19 01:00:01
...,...,...,...,...
372,20210630,65984285,108621415,2021-06-30 12:30:01
373,20210630,5410279,5292425,2021-06-30 12:30:01
374,20210630,29300536,28373063,2021-06-30 12:30:01
375,20210630,2594045,2509931,2021-06-30 12:30:01
